<a href="https://colab.research.google.com/github/IgorBarreto/IgorBarreto/blob/main/DSA/cap06/Notebooks/funcao_janela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Carregue os dados desta meneira


```
!wget - q https://IgorBarreto.github.io/[caminho do arquivo]
```




### configura tamanho da fonte da saida

In [1]:
def increase_font():
  from IPython.display import Javascript
  display(Javascript('''
  for (rule of document.styleSheets[0].cssRules){
    if (rule.selectorText=='body') {
      rule.style.fontSize = '24px'
      break
    }
  }
  '''))
get_ipython().events.register('pre_run_cell', increase_font)
!pip install pyspark --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


### Donload dos dados

In [2]:
# download dos dados

! wget -q https://IgorBarreto.github.io/DSA/cap06/Datasets/vendas.csv
! wget -q https://IgorBarreto.github.io/DSA/cap06/Datasets/2012Q1-capitalbikeshare-tripdata.zip

<IPython.core.display.Javascript object>

### Cria spark session

In [3]:
import pyspark
import zipfile
from pyspark.sql import SparkSession
spark = SparkSession\
  .builder\
  .appName('Sql spark')\
  .getOrCreate()

<IPython.core.display.Javascript object>

### Carrega dados

In [4]:

with zipfile.ZipFile('/content/2012Q1-capitalbikeshare-tripdata.zip') as zip_ref:
   zip_ref.extractall()
tabelas = {'vendas':'vendas.csv',
          'bicicletas_temp':'2012Q1-capitalbikeshare-tripdata.csv'}
for k, v in tabelas.items():
  df=spark.read.csv(v,header=True,inferSchema=True,sep=",")
  df.createOrReplaceTempView(k)
spark.sql(
    """
SELECT
  Duration duracao_segundos,
  `Start date` data_inicio,
  `End date` data_fim,
  `Start station number` numero_estacao_inicio,
  `Start station` estacao_inicio,
  `End station number` numero_estacao_fim,
  `End station` estacao_fim,
  `Bike number` numero_bicileta,
  `Member type` tipo_membro
FROM bicicletas_temp
    """).createOrReplaceTempView('bicicletas')

<IPython.core.display.Javascript object>

In [5]:
SQL="""select * from vendas"""
spark.sql(SQL).show()

<IPython.core.display.Javascript object>

+----------------+----------+-----------+
|nome_funcionario|ano_fiscal|valor_venda|
+----------------+----------+-----------+
|        Romario'|      2020|       2000|
|        Romario'|      2021|       2500|
|        Romario'|      2022|       3000|
|           Zico'|      2020|       1500|
|           Zico'|      2021|       1000|
|           Zico'|      2022|       2000|
|           Pele'|      2020|       2000|
|           Pele'|      2021|       1500|
|           Pele'|      2022|       2500|
+----------------+----------+-----------+



In [6]:
# Total de vendas
SQL="""
select
  ano_fiscal,
  nome_funcionario,
  valor_venda,
  sum(valor_venda) over (partition by ano_fiscal) total_venda_ano
from vendas
order by ano_fiscal"""
spark.sql(SQL).show()

<IPython.core.display.Javascript object>

+----------+----------------+-----------+---------------+
|ano_fiscal|nome_funcionario|valor_venda|total_venda_ano|
+----------+----------------+-----------+---------------+
|      2020|        Romario'|       2000|           5500|
|      2020|           Zico'|       1500|           5500|
|      2020|           Pele'|       2000|           5500|
|      2021|        Romario'|       2500|           5000|
|      2021|           Zico'|       1000|           5000|
|      2021|           Pele'|       1500|           5000|
|      2022|        Romario'|       3000|           7500|
|      2022|           Zico'|       2000|           7500|
|      2022|           Pele'|       2500|           7500|
+----------+----------------+-----------+---------------+



In [7]:
# NÃO FAZ SENTIDO
SQL="""
select
  ano_fiscal,
  nome_funcionario,
  valor_venda,
  sum(valor_venda) total_venda,
  sum(valor_venda) over (partition by ano_fiscal) total_venda_ano
from vendas
group by valor_venda,ano_fiscal,nome_funcionario,valor_venda
order by ano_fiscal"""
spark.sql(SQL).show()

<IPython.core.display.Javascript object>

+----------+----------------+-----------+-----------+---------------+
|ano_fiscal|nome_funcionario|valor_venda|total_venda|total_venda_ano|
+----------+----------------+-----------+-----------+---------------+
|      2020|           Zico'|       1500|       1500|           5500|
|      2020|        Romario'|       2000|       2000|           5500|
|      2020|           Pele'|       2000|       2000|           5500|
|      2021|           Zico'|       1000|       1000|           5000|
|      2021|        Romario'|       2500|       2500|           5000|
|      2021|           Pele'|       1500|       1500|           5000|
|      2022|        Romario'|       3000|       3000|           7500|
|      2022|           Zico'|       2000|       2000|           7500|
|      2022|           Pele'|       2500|       2500|           7500|
+----------+----------------+-----------+-----------+---------------+



In [8]:
#
SQL="""
select
  ano_fiscal,
  nome_funcionario,
  valor_venda,
  sum(valor_venda) over (partition by nome_funcionario) total_venda_ano
from vendas
order by nome_funcionario"""
spark.sql(SQL).show()

<IPython.core.display.Javascript object>

+----------+----------------+-----------+---------------+
|ano_fiscal|nome_funcionario|valor_venda|total_venda_ano|
+----------+----------------+-----------+---------------+
|      2020|           Pele'|       2000|           6000|
|      2021|           Pele'|       1500|           6000|
|      2022|           Pele'|       2500|           6000|
|      2020|        Romario'|       2000|           7500|
|      2021|        Romario'|       2500|           7500|
|      2022|        Romario'|       3000|           7500|
|      2020|           Zico'|       1500|           4500|
|      2021|           Zico'|       1000|           4500|
|      2022|           Zico'|       2000|           4500|
+----------+----------------+-----------+---------------+



In [9]:
# Total de vendas por ano, funcionario e geral
SQL="""
select
  ano_fiscal,
  nome_funcionario,
  valor_venda,
  sum(valor_venda) over (partition by ano_fiscal) total_venda_ano,
  sum(valor_venda) over () total_venda_total

from vendas
order by ano_fiscal"""
spark.sql(SQL).show()

<IPython.core.display.Javascript object>

+----------+----------------+-----------+---------------+-----------------+
|ano_fiscal|nome_funcionario|valor_venda|total_venda_ano|total_venda_total|
+----------+----------------+-----------+---------------+-----------------+
|      2020|        Romario'|       2000|           5500|            18000|
|      2020|           Zico'|       1500|           5500|            18000|
|      2020|           Pele'|       2000|           5500|            18000|
|      2021|        Romario'|       2500|           5000|            18000|
|      2021|           Zico'|       1000|           5000|            18000|
|      2021|           Pele'|       1500|           5000|            18000|
|      2022|        Romario'|       3000|           7500|            18000|
|      2022|           Zico'|       2000|           7500|            18000|
|      2022|           Pele'|       2500|           7500|            18000|
+----------+----------------+-----------+---------------+-----------------+



In [10]:
# Numero de vendas por ano, por funcionario e
# número total de vendas em todos os oanos
SQL="""
select
  ano_fiscal,
  nome_funcionario,
  count(*) over (partition by ano_fiscal,nome_funcionario) total_venda_ano,
  count(*) over () total_de_vendas

from vendas
order by ano_fiscal"""
spark.sql(SQL).show()

<IPython.core.display.Javascript object>

+----------+----------------+---------------+---------------+
|ano_fiscal|nome_funcionario|total_venda_ano|total_de_vendas|
+----------+----------------+---------------+---------------+
|      2020|           Pele'|              1|              9|
|      2020|        Romario'|              1|              9|
|      2020|           Zico'|              1|              9|
|      2021|           Pele'|              1|              9|
|      2021|        Romario'|              1|              9|
|      2021|           Zico'|              1|              9|
|      2022|           Pele'|              1|              9|
|      2022|        Romario'|              1|              9|
|      2022|           Zico'|              1|              9|
+----------+----------------+---------------+---------------+



In [11]:
# Numero de vendas por ano, por funcionario e número total de vendas em todos os oanos usando subquery
SQL="""
select
  ano_fiscal,
  nome_funcionario,
  count(*) over (partition by ano_fiscal,nome_funcionario) total_venda_ano,
  count(*) over () total_de_vendas

from vendas
order by ano_fiscal"""
spark.sql(SQL).show()

<IPython.core.display.Javascript object>

+----------+----------------+---------------+---------------+
|ano_fiscal|nome_funcionario|total_venda_ano|total_de_vendas|
+----------+----------------+---------------+---------------+
|      2020|           Pele'|              1|              9|
|      2020|        Romario'|              1|              9|
|      2020|           Zico'|              1|              9|
|      2021|           Pele'|              1|              9|
|      2021|        Romario'|              1|              9|
|      2021|           Zico'|              1|              9|
|      2022|           Pele'|              1|              9|
|      2022|        Romario'|              1|              9|
|      2022|           Zico'|              1|              9|
+----------+----------------+---------------+---------------+



### Bicicletas

In [12]:
#  data_inicio
#  data_fim
#  numero_estacao_inicio
#  estacao_inicio
#  numero_estacao_fim
#  estacao_fim
#  numero_bicileta
#  tipo_membro
SQL="""select `Start date` from  bicicletas"""
spark.sql(SQL).show()

<IPython.core.display.Javascript object>

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Start date` cannot be resolved. Did you mean one of the following? [`data_fim`, `data_inicio`, `estacao_fim`, `tipo_membro`, `estacao_inicio`].; line 1 pos 7;
'Project ['Start date]
+- SubqueryAlias bicicletas
   +- View (`bicicletas`, [duracao_segundos#58,data_inicio#59,data_fim#60,numero_estacao_inicio#61,estacao_inicio#62,numero_estacao_fim#63,estacao_fim#64,numero_bicileta#65,tipo_membro#66])
      +- Project [Duration#40 AS duracao_segundos#58, Start date#41 AS data_inicio#59, End date#42 AS data_fim#60, Start station number#43 AS numero_estacao_inicio#61, Start station#44 AS estacao_inicio#62, End station number#45 AS numero_estacao_fim#63, End station#46 AS estacao_fim#64, Bike number#47 AS numero_bicileta#65, Member type#48 AS tipo_membro#66]
         +- SubqueryAlias bicicletas_temp
            +- View (`bicicletas_temp`, [Duration#40,Start date#41,End date#42,Start station number#

In [ ]:
# Duração total do aluguel das bikes (em horas)
SQL="""
select
  sum(duracao_segundos)/3600  duracao_horas
from bicicletas"""
spark.sql(SQL).show()

# Duração total do aluguel das bikes (em horas), ao longo do tempo
SQL="""
select
  duracao_segundos,
  sum(duracao_segundos/3600) over(order by data_inicio) duracao__total_horas
from bicicletas"""
spark.sql(SQL).show()

In [ ]:
# Duracao total do aluguel das bikes (em horas), ao longo do tempo, por estação
# de inicio do aluguel da bike, quando a data de inicio é menor '2012-01-08'

SQL="""
select
  row_number() over(order by estacao_inicio,data_inicio) numero_linha,
  estacao_inicio,
  duracao_segundos,
  sum(duracao_segundos/3600) over(partition by estacao_inicio order by data_inicio) duracao__total_horas
from bicicletas
where data_inicio <'2012-01-08'
order by numero_linha
"""
spark.sql(SQL).show(60)

In [ ]:
# Qual é a média de tempo (em horas) de aluguél de bikes da estação ininicio 310177
SQL="""
select
  estacao_inicio,
  duracao_segundos/3600,
  avg(duracao_segundos/3600) over(partition by estacao_inicio order by data_inicio) media_tempo_aluguel
from bicicletas
where numero_estacao_inicio = 31017


"""
spark.sql(SQL).show()
0,09944444444444445
0,5402777777777777

In [ ]:
# Retornar:
# Estação de início, data de início e duração de cada aluguel de bike em segundos
# Duração total de aluguel das bikes ao longo do tempo por estação de início
# Duração média do aluguel de bikes ao longo do tempo por estação de início
# Número de aluguéis de bikes por estação ao longo do tempo
# Somente os registros quando a data de início for inferior a '2012-01-08'


In [ ]:
# Retornar:
# Estação de início, data de início de cada aluguel de bike e duração de cada aluguel em segundos
# Número de aluguéis de bikes (independente da estação) ao longo do tempo
# Somente os registros quando a data de início for inferior a '2012-01-08'

